<center>
    <H1> Introduction à Julia </H1>
<img src="julia.png" width="400"/>
</center>


## Outline

- *Julia* as a language for numerical Computation
- Functional aspects
- Types
- Multiple Dispatch
- Parallelism
- Extensibility

## A language for numerical computation

*Julia*'s  syntax is very similar to langages as *Matlab*, *Python* or *Scilab*, so
switching to *Julia* is fast

- do not require *vectorized* code to run fast using a LLVM-JIT compiler

- it uses references also for function arguments

- indices start to 1 and finish to `end`

- use brackets `[,]` for indexing

- it supports broadcasting, ie. `(1,n)` and `(m, n)` → `(m,n)` and `(m,n)`

- 1D arrays support

- Unicode symbol for notation and some operations 

In [1]:
using LinearAlgebra, Test
function f1(n::Int)
    A=zeros(Int64, n,n);
    for j=1:n ## Fortran Loop Style  !!!!
        for i=1:n
            A[i,j] = i^j
        end
    end
    return A
end    
@elapsed f1(10^4)

1.249269245

In [2]:
function f2(n::Int) 
   A = collect(1:n).^collect(1:n)'
   return A
end
@test f1(3) == f2(3)
@elapsed f2(10^4)

1.243377086

## Functionals aspects
- Anonymous functions : `(x -> x*x)(2) `
- list comprehensions
- supports for `map`, `reduce`, `filter` functions
- functions support variadic arguments (using tuples or names)
- functions are not supposed to modify their arguments, otherwise they follow the `!` convention like `sort!`
- support metaprogramming : like in Lisp "Code is data and data is code"

In [3]:
hilbert(n::Int) = reshape([ 1//(i+j-1) for i=1:n for j=1:n],n,n)
H₃=hilbert(3) 
det(H₃)

1//2160

In [4]:
# seeking integers between squares and cubes
supSquares = filter(x->abs(√(x-1)-round(√(x-1))) < 1e-12, 1:100)
infCubes = filter(x->abs(∛(x+1)-round(∛(x+1))) < 1e-12, 1:100)
ξ = intersect(supSquares, infCubes)

1-element Array{Int64,1}:
 26

## Types
Julia has a hierarchy of types defined by a graph with `Any` as a root

In [5]:
subtypes(Real)

4-element Array{Any,1}:
 AbstractFloat     
 AbstractIrrational
 Integer           
 Rational          

- `x<:y` is the operator for the relation "x is a subtype of y"

In [6]:
Int64 <: Real

true

## Types
some types are abstract (e.g `Real`) → cannot be instantiate

In [7]:
typeof(Real(1.2))

Float64

some types are concrete (e.g. `Int64`) → they have not subtype except `Union{}`

In [8]:
subtypes(Int64)

0-element Array{Type,1}

In [9]:
Union{}<:Int64

true

## Types
### To extend basic types, *Julia* supports
- composite types trough `struct` and `mutable struct`

In [10]:
struct zutos
    x::Int
    a::Array{Int64,1}
end 
w = zutos(2, [1,2])

zutos(2, [1, 2])

In [11]:
w.x = 4

ErrorException: type zutos is immutable

In [12]:
append!(w.a,3)

3-element Array{Int64,1}:
 1
 2
 3

## Types
- tuple types

In [13]:
typeof((3,"haha"))

Tuple{Int64,String}

In [14]:
Tuple{Int64,String} <: Tuple{Real,String}

true

- union types

In [15]:
Real <: Union{String, Real}

true

## Types 
- parametric types 

In [16]:
struct tableau{T<:Number}
    n::Int64
    a::Ptr{T}
end
a = tableau{Int64}(10, 0)

tableau{Int64}(10, Ptr{Int64} @0x0000000000000000)

☢ For efficiency, parametric types are invariants ☢

In [17]:
Array{Int64} <: Array{Real}

false

## Multiple Dispatch
- When Julia executes a function it selects a *method* respect to types of all arguments  : Thus, we can define 
function piecewisely
- no automatic or magic conversions : for operators  arguments are promoted to a common type (user-definable) and use the specific implementation
- let us define `f`

In [18]:
f(x::Float64,y::Float64) = 2x + y
f(x::Int,y::Int) = 2x + y
f(2.,3.) # returns 7.0

7.0

In [19]:
f(2,3)

7

In [20]:
f(2, 3.) 

MethodError: MethodError: no method matching f(::Int64, ::Float64)
Closest candidates are:
  f(::Int64, !Matched::Int64) at In[18]:2
  f(!Matched::Float64, ::Float64) at In[18]:1

- but if we define `g`

In [21]:
    f(x::Real, y::Real) = 2x + y
    f(2.0, 3) 

7.0

In [22]:
methods(f)

# 3 methods for generic function "f":
[1] f(x::Int64, y::Int64) in Main at In[18]:2
[2] f(x::Float64, y::Float64) in Main at In[18]:1
[3] f(x::Real, y::Real) in Main at In[21]:1

## Parallelism
Julia support parallelism trough differents ways
- native distributive parallelism with one master and workers 
- native shared parallelism  → macro `@threads`
- trough MPI.jl could support distributed parallelism
- could support GPU trough packages

### Native distributed parallelism 
- low level routines


In [23]:
using Distributed
addprocs(3)
nprocs()

4

In [24]:
res1 = @spawnat 2 sum(rand(1000,1000).^2)
res2= @spawnat 3 sum(rand(1000,1000).^2)
fetch(res1)+fetch(res2)

667274.1579016141

- high level routines

In [25]:
@everywhere function f3(n::Int)
res = @distributed (+) for i=1:n
      sin(i*π/n)
end
return res
end
@time f3(200000000)

  2.186002 seconds (1.59 M allocations: 80.633 MiB, 0.96% gc time)


1.273239544735062e8

In [26]:
function f4(n::Int)
res =  0
for i=1:n
      res += sin(i*π/n)
end
return res
end
@time f4(200000000)

  3.538700 seconds (29.82 k allocations: 1.578 MiB)


1.2732395447358306e8

## Parallelism
- example with `pmap`

In [27]:
@everywhere using Statistics
@everywhere function cost_mean(x)
   s = zero(eltype(x))
   for i = 1:100
      s += mean(log(sin(exp(xi))) for xi in x)
   end
   return s
end
X = [rand(100_000) for j=1:20];
t1 = @elapsed map(cost_mean, X)
t2 = @elapsed pmap(cost_mean, X)
println("$t1, $t2")

8.881891842, 4.152505259


## Parallelism
- Threads supports is still experimental but works : ⚠️ think to use `export JULIA_NUM_THREADS=4` before launching Julia

In [28]:
using Base.Threads

f(x) = sin(x) + cos(x)

function serial(n)
    s = 0.0
    for x = 1:n
        s += f(x)
    end
    return s
end

@elapsed serial(10^7)

0.672267296

In [29]:
function threads(n)
    res_vec = zeros(nthreads())
    @threads for i ∈ 1:nthreads()
        res_vec[i] = local_sum(threadid(), n, nthreads())
    end
    sum(res_vec)
end

function local_sum(id, n, nthread)
    out = 0.0
    l = 1 + div(n * (id-1), nthread)
    u = div(n * id, nthread)
    for x ∈ l:u
        out += f(x)
    end
    out
end

@elapsed threads(10^7)

0.245307306

## Extensibility
 - *Julia* can be extended trough its trough packages `Pkg.add("MyPackage")` and `using MyPackage`
   - you can use `PyCall` to interact with `Python` → PyPlot package uses this mechanism
   - use MPI ou GPU with packages
 - you can call functions from *C* or *Fortran* using ccall

In [30]:
io = open("skel.c","w")
write(io, "int ajoute2(int x) { return x+2; }")
close(io)
run(`gcc -o ajoute2.so --shared skel.c`);
w = ccall((:ajoute2, "./ajoute2.so"), Int32, (Int32,), 12)
run(`rm ajoute2.so skel.c`)
println("w = $w")

w = 14


## to Conclude
- julia is a *fast* language for doing numerical computations : good replacement for Matlab, R, or Python
- since 2012, the community (numericians and computer scientists) is growing
- In August 2018, the version 1.0 was published
- have a look at https://julialang.org/